# Timeseries EDA + modelling

## Load Data

In [57]:
import pandas as pd


data = pd.read_csv('test_data/SG.csv', sep=';')
data['Time'] = pd.to_datetime(data['Time'], utc=True)

In [58]:
data.dtypes

Time                   datetime64[ns, UTC]
Consumption                        float64
Grid consumption                   float64
Grid backflow                      float64
PV generation                      float64
Battery charging                   float64
Battery discharging                float64
dtype: object

In [59]:
data

,Time,Consumption,Grid consumption,Grid backflow,PV generation,Battery charging,Battery discharging
0,2022-03-05 07:30:00+00:00,64271.779712,59611.639881,0.0,1155.158681,0.0,3461.813704
1,2022-03-05 08:00:00+00:00,73479.560484,68433.723571,0.0,1594.969056,0.0,4079.943209
2,2022-03-05 08:30:00+00:00,83463.478524,72519.254048,0.0,2015.292333,0.0,9322.759101
3,2022-03-05 09:00:00+00:00,77097.835603,71244.783571,0.0,2144.178222,0.0,4177.316192
4,2022-03-05 09:30:00+00:00,80078.206568,71724.207800,0.0,4037.172414,0.0,4617.098633
...,...,...,...,...,...,...,...
1436,2022-04-04 05:30:00+00:00,30475.396429,30475.396429,0.0,0.000000,0.0,0.000000
1437,2022-04-04 06:00:00+00:00,39011.790411,38903.784360,0.0,108.006051,0.0,0.000000
1438,2022-04-04 06:30:00+00:00,38536.480855,38164.752460,0.0,371.728394,0.0,0.000000
1439,2022-04-04 07:00:00+00:00,87121.507746,72062.859365,0.0,1020.296556,0.0,14671.142983


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   Time                 1441 non-null   datetime64[ns, UTC]
 1   Consumption          1437 non-null   float64            
 2   Grid consumption     1437 non-null   float64            
 3   Grid backflow        1437 non-null   float64            
 4   PV generation        1437 non-null   float64            
 5   Battery charging     1437 non-null   float64            
 6   Battery discharging  1437 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 78.9 KB


In [61]:
import plotly.express as px

data_melted = data.melt(id_vars='Time', value_vars=data.drop('Time', axis=1), var_name='Variable', value_name='Value')
fig = px.line(data_melted, x='Time', y='Value', color='Variable')
fig.update_layout(height=800)
fig.show()


## Add Features nad Clean

In [ ]:
# Feature Engineering
# note: just basic features added. Lot of room for improvement
data['day_of_year'] = data['Time'].dt.day_of_year
data['day_of_week'] = data['Time'].dt.day_of_week
data['is_weekend'] = data['day_of_week'] > 4 # 5 for Saturday and 6 for Sunday
data['hours'] = data['Time'].dt.hour + data['Time'].dt.minute / 60

features = ['day_of_year','day_of_week','hours', 'is_weekend']

In [63]:
data.head()

,Time,Consumption,Grid consumption,Grid backflow,PV generation,Battery charging,Battery discharging,day_of_year,day_of_week,is_weekend,hours
0,2022-03-05 07:30:00+00:00,64271.779712,59611.639881,0.0,1155.158681,0.0,3461.813704,64,5,True,7.5
1,2022-03-05 08:00:00+00:00,73479.560484,68433.723571,0.0,1594.969056,0.0,4079.943209,64,5,True,8.0
2,2022-03-05 08:30:00+00:00,83463.478524,72519.254048,0.0,2015.292333,0.0,9322.759101,64,5,True,8.5
3,2022-03-05 09:00:00+00:00,77097.835603,71244.783571,0.0,2144.178222,0.0,4177.316192,64,5,True,9.0
4,2022-03-05 09:30:00+00:00,80078.206568,71724.207800,0.0,4037.172414,0.0,4617.098633,64,5,True,9.5


In [64]:
# clean data
display(data.isna().sum())
display(data[data.isna().any(axis=1)])

# delete incomplete lines
# note: there are better ways handling missing variables, but lets use this one for simplicity
data.dropna(inplace=True)

Time                   0
Consumption            4
Grid consumption       4
Grid backflow          4
PV generation          4
Battery charging       4
Battery discharging    4
day_of_year            0
day_of_week            0
is_weekend             0
hours                  0
dtype: int64

,Time,Consumption,Grid consumption,Grid backflow,PV generation,Battery charging,Battery discharging,day_of_year,day_of_week,is_weekend,hours
657,2022-03-19 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,78,5,True,0.0
658,2022-03-19 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,78,5,True,0.5
659,2022-03-19 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,78,5,True,1.0
660,2022-03-19 01:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,78,5,True,1.5


In [65]:
# handle incomplete days by deleting them
display(data.shape)

samples_per_day = data['day_of_year'].value_counts()
expected_samples = 2*24 # two per hour
days_to_drop = samples_per_day[samples_per_day < expected_samples].index
display(days_to_drop)

data = data[~data['day_of_year'].isin(days_to_drop)]
display(data.shape)

(1437, 11)

Index([78, 64, 94], dtype='int32', name='day_of_year')

(1344, 11)

## Model

In [66]:
# prepare data for Modelling

# split training and evaluation data. 80% train, 20% test
complete_days = data['day_of_year'].unique()
split_idx = int(len(complete_days) * 0.8)
train_days = complete_days[:split_idx]
test_days = complete_days[split_idx:]

data_train = data[data['day_of_year'].isin(train_days)]
data_test = data[data['day_of_year'].isin(test_days)]

target = 'Consumption'

X_train = data_train[features]
y_train = data_train[target]
X_test = data_test[features]
y_test = data_test[target]


In [67]:
# model

from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

model = RandomForestRegressor(random_state=2)
#model = GradientBoostingRegressor(random_state=2)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
accuracy_percentage = r2 * 100
print(f"The model is {accuracy_percentage:.2f}% accurate.")


result = pd.DataFrame()
result['Time'] = data_test['Time']
result['Measured'] = y_test
result['Predicted'] = y_pred
result

Mean Squared Error: 148968029.74126828
The model is 74.70% accurate.


,Time,Measured,Predicted
1137,2022-03-29 00:00:00+00:00,50338.026437,45178.979947
1138,2022-03-29 00:30:00+00:00,28712.092619,41438.634359
1139,2022-03-29 01:00:00+00:00,26963.064048,37906.431155
1140,2022-03-29 01:30:00+00:00,27233.657302,30690.338110
1141,2022-03-29 02:00:00+00:00,26163.666468,29272.950397
...,...,...,...
1420,2022-04-03 21:30:00+00:00,28300.509683,28654.310078
1421,2022-04-03 22:00:00+00:00,28574.806363,28328.824420
1422,2022-04-03 22:30:00+00:00,28247.599286,28765.010734
1423,2022-04-03 23:00:00+00:00,29366.549246,28279.156292


In [68]:
# plot results
data_melted = result.melt(id_vars='Time', value_vars=result, var_name='Variable', value_name='Value')
fig = px.line(data_melted, x='Time', y='Value', color='Variable', title=target, line_shape='linear')
for trace in fig.data:
    trace.update(connectgaps=False)
fig.update_layout(height=800)
fig.show()